In [8]:
from ase.neighborlist import NeighborList
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
import json, os
import numpy as np
from ase.visualize import view

In [14]:
structures_file = '/Users/myless/Packages/structure_maker/structures_V111_Cr3_Ti3_Zr2_W5_Ta1.json'
#entries = json.load(open('/Users/myless/Packages/structure_maker/V111_Cr3_Ti3_Zr2_W5_Ta1_entries.json'))
entries = json.load(open('/home/myless/Packages/structure_maker/V111_Cr3_Ti3_Zr2_W5_Ta1_entries.json','r'))
#entries = json.load(open(structures_file,'r'))
print(type(entries[0]))

<class 'dict'>


In [15]:
#structure = Structure.from_dict(entries[0]['structure'])
structure = Structure.from_dict(entries[1]['structure'])
atoms = AseAtomsAdaptor.get_atoms(structure)

In [16]:
view(atoms, viewer='x3d')

In [17]:
def return_x_neighbors(atoms, target_atom_index, x_neighbor, alat, tolerance = 1.05):
    """
    Returns the indices and distances of the nearest neighbors of a target atom.
    
    Parameters:
    atoms (Atoms): The atomic structure.
    target_atom_index (int): The index of the target atom.
    x_neighbor (int): The number of neighbors to return (1, 2, or 3).
    alat (float): The lattice constant.
    tolerance (float, optional): The tolerance factor for the cutoff distance. Default is 1.05.
    
    Returns:
    nearest_neighbors (list): The indices of the nearest neighbors.
    distances (list): The distances to the nearest neighbors.
    """

    # get the cutoff distance for the nearest neighbors
    if x_neighbor == 1:
        cutoff = alat * np.sqrt(3)/2 * tolerance # 5% larger than the nearest neighbor distance for tolerance
    elif x_neighbor == 2:
        cutoff = alat * tolerance 
    elif x_neighbor == 3:
        cutoff = alat * np.sqrt(2) * tolerance
    else:
        print('x_neighbor must be 1, 2, or 3')
        return None
    
    # create the neighbor list 
    nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
    nl.update(atoms)
    indices, offsets = nl.get_neighbors(target_atom_index)
    distances = [atoms.get_distance(target_atom_index, i, mic=True) for i in indices]

    # get the number of indices based on if we need the nearest, next-nearest, or next-next-nearest neighbors
    if x_neighbor == 1:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[:8]
    elif x_neighbor == 2:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[8:14]
    elif x_neighbor == 3:
        sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[14:26]
    else:
        print('x_neighbor must be 1, 2, or 3')
        return None
    
    # get the indices and distances of the nearest neighbors sorted by distance
    nearest_neighbors = [index for index,_ in sorted_neighbors]
    distances = [distance for _, distance in sorted_neighbors]
    return nearest_neighbors, distances

def return_next_nearest_neighbors(atoms, target_atom_index, alat, tolerance = 1.05):
    cutoff = alat * tolerance # 5% larger than the nearest neighbor distance for tolerance
    nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
    nl.update(atoms)
    indices, offsets = nl.get_neighbors(target_atom_index)
    distances = [atoms.get_distance(target_atom_index, i, mic=True) for i in indices]
    sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[8:14]
    nearest_neighbors = [index for index,_ in sorted_neighbors]
    distances = [distance for _, distance in sorted_neighbors]
    return nearest_neighbors, distances

def return_next_next_nearest_neighbors(atoms, target_atom_index, alat, tolerance = 1.05):
    cutoff = alat * np.sqrt(2) * tolerance # 5% larger than the nearest neighbor distance for tolerance
    nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
    nl.update(atoms)
    indices, offsets = nl.get_neighbors(target_atom_index)
    distances = [atoms.get_distance(target_atom_index, i, mic=True) for i in indices]
    sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])[14:26]
    nearest_neighbors = [index for index,_ in sorted_neighbors]
    distances = [distance for _, distance in sorted_neighbors]
    return nearest_neighbors, distances

In [18]:
nearest_neighbors, distances = return_x_neighbors(atoms = atoms, target_atom_index=0, x_neighbor = 1, alat = 3.01)
nn_neighbors, distances = return_x_neighbors(atoms = atoms, target_atom_index=0, x_neighbor = 2, alat = 3.01)
nnn_neighbors, distances = return_x_neighbors(atoms = atoms, target_atom_index=0, x_neighbor = 3, alat = 3.01)
print("Nearest : ", nearest_neighbors)
print("Next-nearest : ", nn_neighbors)
print("Next-next-nearest : ", nnn_neighbors)

Nearest :  [3, 2, 1, 34, 114, 20, 24, 56]
Next-nearest :  [51, 8, 35, 38, 104, 39]
Next-next-nearest :  [120, 5, 103, 43, 90, 74, 77, 99, 7, 40, 52, 100]


In [25]:
cutoff = 3.01*2
target_atom_index = 0 
target_atom = atoms[target_atom_index]
nl = NeighborList([cutoff/2]*len(atoms), self_interaction=False, bothways=True)
nl.update(atoms)
indices, offsets = nl.get_neighbors(target_atom_index)

In [26]:
distances = [atoms.get_distance(target_atom_index, i, mic=True) for i in indices]
sorted_neighbors = sorted(zip(indices, distances), key=lambda x: x[1])

In [6]:
# for the nearest neighbors
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(nearest_neighbors)
print(distances)

[2, 1, 20, 56, 3, 114, 34, 24]
[2.528781690578451, 2.5518681647023507, 2.5827391589853668, 2.5948190295609246, 2.601436650455199, 2.601705577190455, 2.6370253895756517, 2.6905205780482393]


In [31]:
cutoff = 3.01 * np.sqrt(3)/2
largest = 2.6905205780482393
print(largest/cutoff)

nn_cutoff = 3.01 
nn_largest = 3.0708511061670936
print(nn_largest/nn_cutoff)

nnn_cutoff = 3.01 * np.sqrt(2)
nnn_largest = 4.330260074527209
print(nnn_largest/nnn_cutoff)

1.0321413820573944
1.020216314341227
1.0172612169433735


In [15]:
# for the nearest neighbors with new cutoff
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(nearest_neighbors)
print(distances)

[2, 1, 20, 56, 3, 114, 34, 24]
[2.528781690578451, 2.5518681647023507, 2.5827391589853668, 2.5948190295609246, 2.601436650455199, 2.601705577190455, 2.6370253895756517, 2.6905205780482393]


In [18]:
# for the next nearest neighbors
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(nearest_neighbors)
print(distances)

[39, 35, 38, 8]
[2.9263286606110857, 3.0156507502991974, 3.042016814987278, 3.0455899692119703]


In [21]:
# for the next next nearest neighbors
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(nearest_neighbors)
print(distances)

[51, 104, 77, 5]
[3.0600891385899285, 3.0708511061670936, 4.170950226833095, 4.222201502310656]


In [24]:
# for all neighbors in cutoff
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(f"There are {len(nearest_neighbors)} neighbors in the cutoff")
print(nearest_neighbors)
print(distances)

There are 26 neighbors in the cutoff
[2, 1, 20, 56, 3, 114, 34, 24, 39, 35, 38, 8, 51, 104, 77, 5, 52, 74, 120, 40, 7, 99, 100, 90, 43, 103]
[2.528781690578451, 2.5518681647023507, 2.5827391589853668, 2.5948190295609246, 2.601436650455199, 2.601705577190455, 2.6370253895756517, 2.6905205780482393, 2.9263286606110857, 3.0156507502991974, 3.042016814987278, 3.0455899692119703, 3.0600891385899285, 3.0708511061670936, 4.170950226833095, 4.222201502310656, 4.223651969203701, 4.238968555654315, 4.247681320022284, 4.2551695056287615, 4.261501640804096, 4.263666452877288, 4.275731765940114, 4.284326440788527, 4.290612057853512, 4.330260074527209]


In [27]:
# for all neighbors in cutoff
nearest_neighbors = [index for index,_ in sorted_neighbors]
distances = [distance for _, distance in sorted_neighbors]
print(f"There are {len(nearest_neighbors)} neighbors in the cutoff")
print(nearest_neighbors)
print(distances)

There are 84 neighbors in the cutoff
[2, 1, 20, 56, 3, 114, 34, 24, 39, 35, 38, 8, 51, 104, 77, 5, 52, 74, 120, 40, 7, 99, 100, 90, 43, 103, 93, 113, 57, 15, 16, 61, 25, 23, 53, 59, 55, 32, 69, 110, 89, 111, 70, 124, 21, 105, 18, 85, 84, 60, 9, 78, 4, 10, 13, 6, 98, 19, 44, 48, 68, 37, 36, 31, 62, 96, 94, 97, 88, 109, 83, 11, 14, 91, 64, 12, 79, 73, 17, 86, 80, 28, 123, 33]
[2.528781690578451, 2.5518681647023507, 2.5827391589853668, 2.5948190295609246, 2.601436650455199, 2.601705577190455, 2.6370253895756517, 2.6905205780482393, 2.9263286606110857, 3.0156507502991974, 3.042016814987278, 3.0455899692119703, 3.0600891385899285, 3.0708511061670936, 4.170950226833095, 4.222201502310656, 4.223651969203701, 4.238968555654315, 4.247681320022284, 4.2551695056287615, 4.261501640804096, 4.263666452877288, 4.275731765940114, 4.284326440788527, 4.290612057853512, 4.330260074527209, 4.9223701603211785, 4.941624869698045, 4.946408494353676, 4.948395167287861, 4.9587699439739845, 4.958862467081346, 4

In [28]:
neighbors = [2.528781690578451, 2.5518681647023507, 2.5827391589853668, 2.5948190295609246, 2.601436650455199, 2.601705577190455, 2.6370253895756517, 2.6905205780482393, 2.9263286606110857, 3.0156507502991974, 3.042016814987278, 3.0455899692119703, 3.0600891385899285, 3.0708511061670936, 4.170950226833095, 4.222201502310656, 4.223651969203701, 4.238968555654315, 4.247681320022284, 4.2551695056287615, 4.261501640804096, 4.263666452877288, 4.275731765940114, 4.284326440788527, 4.290612057853512, 4.330260074527209]
print(len(neighbors))


26


In [7]:

visualization_supercell = atoms.copy()

for atom in visualization_supercell:
    if atom.index == target_atom_index:
        atom.symbol = 'Xe'
    elif atom.index in nearest_neighbors:
        atom.symbol = 'Kr'

view(visualization_supercell,viewer='ngl')

NameError: name 'target_atom_index' is not defined

In [10]:
v=_
print(dir(v.view))

['__annotations__', '__class__', '__copy__', '__deepcopy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_active_widgets', '_add_notifiers', '_add_shape', '_all_trait_default_generators', '_call_widget_constructed', '_camera_orientation', '_camera_str', '_clear_component_auto_completion', '_comm_changed', '_compare', '_control_comm', '_coordinates_dict', '_create_ibtn_fullscreen', '_create_player', '_cross_validation_lock', '_default_keys', '_descriptors', '_display_image', '_display_repr', '_dom_classes', '_dry_run', '_event', '_execute_js_code', '_fire_callbacks', '_gen_repr_from_keys', '_get_embed_state', '_get_full_params', '_get_

In [16]:
prim = structure.get_primitive_structure()
print(prim)

Full Formula (Zr2 Ta1 Ti3 V111 Cr3 W5)
Reduced Formula: Zr2TaTi3V111Cr3W5
abc   :  13.033682  13.033682  13.033682
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (125)
  #  SP           a         b         c    tags
---  ----  --------  --------  --------  ------
  0  V     0.800406  0.001593  0.002073       0
  1  V     0.602568  0.999584  0.997847       0
  2  V     0.795845  0.194168  0.00246        0
  3  V     0.80172   0.000345  0.201677       0
  4  V     0.400092  0.998784  0.000226       0
  5  Ti    0.601038  0.198962  0.99898        0
  6  V     0.800429  0.401747  0.999564       0
  7  V     0.595129  0.993995  0.197162       0
  8  V     0.803063  0.202885  0.208094       0
  9  V     0.792514  0.99785   0.393446       0
 10  V     0.199498  0.999408  0.000292       0
 11  V     0.399416  0.199218  0.998505       0
 12  V     0.599013  0.400748  0.99864        0
 13  V     0.797859  0.599204  0.999347       0
 14  V     0.402552  0.

In [24]:
import numpy as np 
alat = 3.01
site = structure[5]
n_dist = alat * np.sqrt(3)/2
structure.get_neighbors(site=site,r=n_dist)

[PeriodicNeighbor: Cr (3.073, 9.048, -3.011) [0.4011, 0.004117, 0.8054],
 PeriodicNeighbor: V (2.994, 8.988, -0.05749) [0.5934, 0.1951, 0.7962],
 PeriodicNeighbor: V (2.971, 12.05, -2.978) [0.6026, -0.000416, 0.9978],
 PeriodicNeighbor: V (3.016, 12.07, -0.09366) [0.7958, 0.1942, 1.002],
 PeriodicNeighbor: V (6.005, 12.03, -0.005522) [0.7992, 0.3987, 1.199]]

AttributeError: 'Structure' object has no attribute 'get_all_neighbors_'

In [7]:
cutoffs = []
nl = NeighborList(cutoffs)

Atoms(symbols='Cr3TaTi3V111W5Zr2', pbc=True, cell=[[-7.5249999999999995, 7.5249999999999995, 7.5249999999999995], [7.5249999999999995, -7.5249999999999995, 7.5249999999999995], [7.5249999999999995, 7.5249999999999995, -7.5249999999999995]], tags=...)
